In [1]:
import pandas as pd
import numpy as np

import scipy
from scipy import stats
import statsmodels as sm
from statsmodels.regression.linear_model import OLS

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline

from datetime import datetime
import time
import datetime

import eikon as ek
#API Key Lionel: d505f265d9bd4c0cb7df44481ece31a0ad8d1974
#API Key Raphi: f5a341b614d945b9b93a71fe53348ba119d11e99
ek.set_app_key('d505f265d9bd4c0cb7df44481ece31a0ad8d1974')

Import Data from Eikon API:

In [2]:
Makro_Data = ['USUNR=ECI',  #US Unemploymet
              'USCPI=ECI',  #US CPI
              'USPMI=ECI',  #US PMI
              'EUUNR=ECI',  #EU Unemployment
              'EUHICY=ECI', #EU CPI
              'EUPMI=ECI',  #EU PMI            
              'CHJOB=ECI',  #CH Unemployment
              'CHCPIY=ECI', #CH CPI
              'CHPMI=ECI',  #CH PMI
              'CNURUA=ECI', #China Unemployment  
              'CNCPI=ECI',  #China CPI
              'CNPMIB=ECI', #China PMI
              'RUUNR=ECI',  #Russia Unemployment
              'RUCPIY=ECI', #Russia CPI
              'RUPMIM=ECI', #Russia PMI          
             ]  

Futures = ['CLc1',      #NYMEX Light Sweet Crude Oil (WTI)
           'NGLNMc1',   #Intercontinetanl Exchange UK NBP Natrual Gas
           'NGc1',      #NYMEX Henry Hub Natural Gas
           'GCc1',      #COMEX Gold
           'SIc1',      #COMEX Silver
           'PLc1',      #NYM Platinum
           'PAc1',      #CYM Palladium
           'Wc1',       #CBoT Wheat
           'Cc1',       #CBoT Corn
           'LCc1',      #CME Live Cattle
           'OJc1'       #ICE Orange Juice
          ]





In [73]:
macro_raw = ek.get_timeseries(Makro_Data,
                          start_date='2012-01-01',
                          end_date='2022-01-01',
                          fields='VALUE',
                          interval='monthly')  

macro_raw.columns = macro_raw.columns.str.replace("=", "_")
macro_raw = macro_raw.unstack()
macro_raw = macro_raw.reset_index()
macro_raw = macro_raw.rename(columns={'VALUE': 'Macro_Variable', 0: 'Value'})
macro_raw['Date'] = pd.to_datetime(macro_raw['Date']).dt.date
liste_SQL_macro = macro_raw.values.tolist()
macro_raw.head(5)


,Macro_Variable,Date,Value
0,USUNR_ECI,2012-01-31,8.3
1,USUNR_ECI,2012-02-29,8.3
2,USUNR_ECI,2012-03-31,8.2
3,USUNR_ECI,2012-04-30,8.2
4,USUNR_ECI,2012-05-31,8.2


In [88]:
futures_raw = pd.DataFrame()
try:
    itemName=Futures
    dateStart="2011-12-31"
    dateEnd="2022-01-01"

    #Create Loop to inport all Futures:
    for y in range(0,len(Futures)):
        data_input = ek.get_timeseries(Futures[y],fields=["Close"], start_date = dateStart , 
                                    end_date = dateEnd , interval='monthly', corax = 'adjusted' )
        data_input['Instrument'] = Futures[y]
        futures_raw = futures_raw.append(data_input)
except ek.EikonError as ex:
    print('Eikon Error')
    print(ex)
except Exception as ex:
    print(ex)

futures_raw = futures_raw.sort_values(['Instrument','Date'])
futures_raw["Return"] = futures_raw.groupby("Instrument")['CLOSE'].apply(lambda x: np.log(x) - np.log(x.shift(1)))
futures_raw = futures_raw.reset_index()
futures_raw['Date'] = pd.to_datetime(futures_raw['Date']).dt.date
liste_SQL_futures = futures_raw.values.tolist()
futures_raw

OJc1,Date,CLOSE,Instrument,Return
0,2011-12-31,98.83,CLc1,NaN
1,2012-01-31,98.48,CLc1,-0.003548
2,2012-02-29,107.07,CLc1,0.083629
3,2012-03-31,103.02,CLc1,-0.038560
4,2012-04-30,104.87,CLc1,0.017798
...,...,...,...,...
1326,2021-08-31,706.75,Wc1,0.004254
1327,2021-09-30,725.50,Wc1,0.026184
1328,2021-10-31,772.75,Wc1,0.063095
1329,2021-11-30,773.75,Wc1,0.001293


In [89]:
import sqlite3
from sqlite3 import Error
 
""" create a database connection tool to a SQLite database """
def create_connection(db_file):
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
    finally:
        if conn:
            conn.close()

if __name__ == '__main__':
    create_connection(r"C:\Users\lione\OneDrive\Dokumente\GitHub\AQM\SQL_DB.db")

2.6.0


In [90]:
conn = sqlite3.connect('SQL_DB.db')
c = conn.cursor()
c.execute('''DROP TABLE IF EXISTS MACRO_RAW;''')
c.execute('''DROP TABLE IF EXISTS FUTURES_RAW;''')

In [91]:
c.execute('''CREATE TABLE MACRO_RAW (
    "Macro_Variable" TEXT,
    DATE DATETIME,
	"Value" NUMERIC
);''')

c.execute('''CREATE TABLE FUTURES_RAW (
    DATE DATETIME,
	"CLOSE" NUMERIC,
    "Instrument" TEXT,
    "Return" NUMERIC
);''')


In [92]:
conn = sqlite3.connect(r"C:\Users\lione\OneDrive\Dokumente\GitHub\AQM\SQL_DB.db")
c = conn.cursor()
c.executemany("INSERT INTO MACRO_RAW(Macro_Variable, DATE, Value) VALUES (?,?,?)", liste_SQL_macro)    
conn.commit()

c.executemany("INSERT INTO FUTURES_RAW(DATE, CLOSE, Instrument, Return) VALUES (?,?,?,?)", liste_SQL_futures)    
conn.commit()

In [93]:
#Select Statement to import the SQL Data back into Python:   
c = conn.cursor()
c.execute("SELECT * FROM MACRO_RAW")
macro_sql_data=c.fetchall()
macro_raw_sql = pd.DataFrame(macro_sql_data,columns=['Macro_Variable', 'DATE', 'Value'])
macro_raw_sql = macro_raw_sql.set_index('DATE')
macro_raw_sql

#Select Futures Data from SQL DB:   
c.execute("SELECT * FROM FUTURES_RAW")
futures_sql_data=c.fetchall()
futures_raw_sql = pd.DataFrame(futures_sql_data,columns=['DATE', 'CLOSE', 'Instrument', 'Return'])
futures_raw_sql = futures_raw_sql.set_index('DATE')
futures_raw_sql

,CLOSE,Instrument,Return
DATE,,,
2011-12-31,98.83,CLc1,NaN
2012-01-31,98.48,CLc1,-0.003548
2012-02-29,107.07,CLc1,0.083629
2012-03-31,103.02,CLc1,-0.038560
2012-04-30,104.87,CLc1,0.017798
...,...,...,...
2021-08-31,706.75,Wc1,0.004254
2021-09-30,725.50,Wc1,0.026184
2021-10-31,772.75,Wc1,0.063095


In [97]:
macro_raw_pivot = macro_raw_sql.pivot(columns='Macro_Variable', values=['Value'])
macro_raw_pivot = macro_raw_pivot.droplevel(level=0, axis=1)
macro_raw_pivot.to_excel('Files\Macro_raw_pivot.xlsx')
macro_raw_pivot

futures_raw_close_pivot = futures_raw_sql.pivot(columns='Instrument', values=['CLOSE'])
futures_raw_close_pivot = futures_raw_close_pivot.droplevel(level=0, axis=1)
futures_raw_close_pivot.to_excel('Files\Futures_raw_CLOSE_pivot.xlsx')
futures_raw_close_pivot

futures_raw_returns_pivot = futures_raw_sql.pivot(columns='Instrument', values=['Return'])
futures_raw_returns_pivot = futures_raw_returns_pivot.droplevel(level=0, axis=1)
futures_raw_returns_pivot.to_excel('Files\Futures_raw_returns_pivot.xlsx')
futures_raw_returns_pivot


Instrument,CLc1,Cc1,GCc1,LCc1,NGLNMc1,NGc1,OJc1,PAc1,PLc1,SIc1,Wc1
DATE,,,,,,,,,,,
2011-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-01-31,-0.003548,-0.011669,0.104223,0.015742,0.035098,-0.177449,0.217209,0.045254,0.124642,0.175813,0.020095
2012-02-29,0.083629,0.027018,-0.016185,0.022179,0.054151,0.044156,-0.097193,0.029731,0.064479,0.039819,-0.002631
2012-03-31,-0.038560,-0.019224,-0.024031,-0.058057,0.035080,-0.207404,-0.147004,-0.078729,-0.031720,-0.063076,-0.005283
2012-04-30,0.017798,0.024920,-0.003541,-0.014214,-0.060838,0.072124,-0.155215,0.042856,-0.044050,-0.047622,-0.019871
...,...,...,...,...,...,...,...,...,...,...,...
2021-08-31,-0.076555,-0.024053,0.001323,-0.025932,0.207778,0.111804,0.011963,-0.073216,-0.032029,-0.063390,0.004254
2021-09-30,0.091054,0.005137,-0.033446,0.013569,0.676408,0.292980,-0.015353,-0.260308,-0.053689,-0.084389,0.026184
2021-10-31,0.107797,0.057029,0.015658,0.054864,-0.414303,-0.078141,-0.096178,0.042952,0.059295,0.083763,0.063095
